# Cleaning Up

This notebook demonstrates how to clean up all the resources created in the previous set of notebooks.

In [1]:
import boto3
import json

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [3]:
%store -r

Please note these steps rely on each step being fully completed before the other will work. If you receive an error, wait a minute or two and try again.

## Clean up Campaigns

Start by cleaning up the campaigns. You need to delete campaigns before you can delete the solution versions upon which they are based. 

The code below will delete the Campaigns you created.

In [4]:
personalize.delete_campaign(
    campaignArn = userpersonalization_campaign_arn
)
personalize.delete_campaign(
    campaignArn = sims_campaign_arn
)
personalize.delete_campaign(
    campaignArn = rerank_campaign_arn
)

{'ResponseMetadata': {'RequestId': 'c3a284b5-eded-4c10-988e-8d505fbd060e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 21:30:06 GMT',
   'x-amzn-requestid': 'c3a284b5-eded-4c10-988e-8d505fbd060e',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up event trackers

Next, clean up any event trackers. The code below will delete your event tracker.

In [5]:
personalize.delete_event_tracker(
    eventTrackerArn = event_tracker_arn
)

{'ResponseMetadata': {'RequestId': 'aeebb5af-c507-4599-868a-3d7c7b982ec7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 21:30:09 GMT',
   'x-amzn-requestid': 'aeebb5af-c507-4599-868a-3d7c7b982ec7',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up Filters

Next, clean up any filters. The code below iterate over all filters you created and delete them. Note, if you get an error about a campaign, that is fine, wait for 2 minutes then try the cell below again. 

Note: You will not be able to delete the filters until the campaign deletion completes, which may take a few minutes

In [11]:
personalize.delete_filter(filterArn = genre_filter_arn)

for filter_arn in interaction_filter_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )
for filter_arn in meta_filter_decade_arns:
    personalize.delete_filter(
        filterArn = filter_arn
    )

## Clean up solutions

Next, clean up the solutions. The code below will delete the solutions that were created.

In [12]:
personalize.delete_solution(
    solutionArn = user_personalization_solution_arn
)
personalize.delete_solution(
    solutionArn = sims_solution_arn
)
personalize.delete_solution(
    solutionArn = rerank_solution_arn
)

{'ResponseMetadata': {'RequestId': '6a54d813-f52b-4615-ba5a-a93b15121b25',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 21:36:14 GMT',
   'x-amzn-requestid': '6a54d813-f52b-4615-ba5a-a93b15121b25',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up datasets

Next, clean up the datasets. The code below will delete the item and interactions datasets.

In [13]:
personalize.delete_dataset(
    datasetArn = items_dataset_arn
)
personalize.delete_dataset(
    datasetArn = interactions_dataset_arn
)

{'ResponseMetadata': {'RequestId': 'ab4a821e-6aab-4f4f-90ec-11436aa0f5ee',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 21:36:18 GMT',
   'x-amzn-requestid': 'ab4a821e-6aab-4f4f-90ec-11436aa0f5ee',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the schemas

Next, clean up the schemas. You created an interaction and item schema, delete them.

Note: You will not be able to delete the schemas until the dataset deletion completes, which may take a few minutes

In [15]:
personalize.delete_schema(
    schemaArn = interaction_schema_arn
)
personalize.delete_schema(
    schemaArn = itemmetadataschema_arn
)

{'ResponseMetadata': {'RequestId': '0541a1ce-18af-4034-800e-740f6f4566c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 21:37:05 GMT',
   'x-amzn-requestid': '0541a1ce-18af-4034-800e-740f6f4566c1',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the dataset groups

Finally, clean up the dataset group:

In [16]:
personalize.delete_dataset_group(
    datasetGroupArn = dataset_group_arn
)

{'ResponseMetadata': {'RequestId': '2d4d73de-c6a3-47b6-ba5b-28c489ed514f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 01 Feb 2021 21:37:08 GMT',
   'x-amzn-requestid': '2d4d73de-c6a3-47b6-ba5b-28c489ed514f',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Clean up the S3 bucket and IAM role

Start by deleting the role, then empty the bucket, then delete the bucket.

In [17]:
iam = boto3.client('iam')

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [18]:
iam.list_attached_role_policies(
    RoleName = role_name
)

{'AttachedPolicies': [{'PolicyName': 'AmazonS3FullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/AmazonS3FullAccess'},
  {'PolicyName': 'AmazonPersonalizeFullAccess',
   'PolicyArn': 'arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess'}],
 'IsTruncated': False,
 'ResponseMetadata': {'RequestId': '7f1725a9-e020-4a72-ab34-4168f570f1c4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7f1725a9-e020-4a72-ab34-4168f570f1c4',
   'content-type': 'text/xml',
   'content-length': '728',
   'date': 'Mon, 01 Feb 2021 21:37:12 GMT'},
  'RetryAttempts': 0}}

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [19]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
)

{'ResponseMetadata': {'RequestId': 'd2042331-2e40-461e-bde1-8c0daf6c1be6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd2042331-2e40-461e-bde1-8c0daf6c1be6',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 01 Feb 2021 21:37:14 GMT'},
  'RetryAttempts': 0}}

In [20]:
iam.detach_role_policy(
    RoleName = role_name,
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

{'ResponseMetadata': {'RequestId': '5d6f90bc-202d-4af1-aed5-14ac39664e34',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5d6f90bc-202d-4af1-aed5-14ac39664e34',
   'content-type': 'text/xml',
   'content-length': '212',
   'date': 'Mon, 01 Feb 2021 21:37:16 GMT'},
  'RetryAttempts': 0}}

Finally, you should be able to delete the IAM role.

In [21]:
iam.delete_role(
    RoleName = role_name
)

{'ResponseMetadata': {'RequestId': '0a6392f1-f15c-425a-ac59-800525695792',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0a6392f1-f15c-425a-ac59-800525695792',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Mon, 01 Feb 2021 21:37:19 GMT'},
  'RetryAttempts': 0}}

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.